Answer all questions and submit them either as an IPython notebook, LaTeX document, or Markdown document. Each question is worth 25 points.

This homework is due Friday, October 9, 2015.

In [ ]:
%matplotlib inline
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np

## Question 1

The data below provides counts of a flour beetle (Tribolium confusum) population at various points in time:

In [ ]:
days = 0,8,28,41,63,79,97,117,135,154
beetles = 2,47,192,256,768,896,1120,896,1184,1024

plt.plot(days, beetles)

An elementary model for population growth is the logistic model:

$$\frac{dN}{dt} = rN\left(1 - \frac{N}{K}\right)$$

where $N$ is population size, $t$ is time, $r$ is a growth rate parameter, and $K$ is a parameter that represents the population carrying capacity of the environment. The solution to this differential equation is given by: 

$$N_t = f(t) = \frac{KN_0}{N_0 + (K - N_0)\exp(-rt)}$$

where $N_t$ denotes the population size at time $t$. 

1) Fit the logistic growth model to the flour beetle data using optimization to minimize the sum of squared errors between model predictions and observed counts.

In [ ]:
from scipy.optimize import minimize
import math
def calcError(args):
    error = 0;
    for i in range(len(days)):
        calcBeetles = f([args[0],beetles[0],args[1],days[i]])
        error += (calcBeetles-beetles[i])**2
    return error

def f(args):
    #K N0 r t
    calcBeetles = (args[0]*args[1])/(args[1]+(args[0]-args[1])*math.exp(-args[2]*args[3]))
    return calcBeetles


result = minimize(calcError, [1700,0.4],method = 'Nelder-Mead')
K = result.x[0]
r = result.x[1]
calcBeetles = np.zeros([len(days),1])
for i in range(len(days)):
    calcBeetles[i] = f([K,beetles[0],r,days[i]])

print(result)
plt.plot(days,calcBeetles)
plt.plot(days,beetles,'ro')
print(calcBeetles)

2) In many population modeling applications, an assumption of lognormality is adopted. The simplest assumption would be that the $\log(N_t)$ are independent and normally distributed with mean $\log[f(t)]$ and variance $\sigma^2$. Find the MLEs under this assumption, and provide estimates of standard errors and correlation between them.

In [ ]:
from scipy.stats import norm
days_new = np.copy(days)
beetles_new = np.copy(beetles)

def calcLikelihood(args):
    likelyhood = 0
    for i in range(len(beetles_new)):
        logft = f([args[0],beetles_new[0],args[1],days_new[i]])
        likelyhood += (-1*norm.logpdf(np.log(beetles_new[i]),loc=logft,scale=args[2]))
    return likelyhood

def f(args):
    #K N0 r t
    try:
        calcBeetles = np.log((args[0]*args[1])/(args[1]+(args[0]-args[1])*np.exp(-args[2]*args[3])))
    except OverflowError:
        print("Error")
    return calcBeetles

logBeetles_new = np.zeros([len(beetles),1])

nTimes =  10
K = np.zeros(nTimes)
r = np.zeros(nTimes)
var = np.zeros(nTimes)
length = len(beetles_new)
counter = 0
for i in range(nTimes):
    index = np.random.randint(0,length-1,length)
    for t in range(length):
        beetles_new[t] = np.copy(beetles[index[t]])
        days_new[t] = np.copy(days[index[t]])
    for t in range(len(beetles_new)):
        logBeetles_new[t] = math.log(beetles_new[t])
    var0 = np.var(logBeetles_new)
    result = minimize(calcLikelihood, [1700,0.4,var0],method = 'Nelder-Mead',options = {'xtol':1e-8, 'disp' : False})
    #print(result)
    if result.success:
        print('New K',result.x[0],'new r',result.x[1],'New var',result.x[2])
        K[counter] = result.x[0]
        r[counter] = result.x[1]
        var[counter] = result.x[2]
        counter+=1
K_final = np.delete(K,range(counter,len(K)))
r_final = np.delete(r,range(counter,len(K)))
var_final = np.delete(var,range(counter,len(K)))


print('variance of K',np.var(K_final))
print('variance of r', np.var(r_final))
print('variance of variance',np.var(var_final))

## Question 2

1)Implement simulated annealing for minimizing the AIC for the baseball salary regression problem. Model your algorithm on the example given in class. 
    

In [ ]:
aic = lambda g: g.nobs * np.log((g.resid**2).sum()/g.nobs) + 2*len(g.beta)

def plotResults(aic_values,aic_best,solution_best):

    plt.plot(aic_values)
    plt.xlim(0, len(aic_values))
    plt.xlabel('Iteration')
    plt.ylabel('AIC')
    print('Best AIC: {0}\nBest solution: {1}\nDiscovered at iteration {2}'.format(aic_best,
                np.where(solution_best==True),
                np.where(aic_values==aic_best)[0][0]))
    plt.plot(np.where(aic_values==aic_best)[0][0], aic_best, 'ro')
    plt.show()

def simAnnealing(periods,cooling,tau_start,numChangeNeigh):
    baseball = pd.read_table("/Users/Ahmet/Box Sync/Classes/Vanderbilt/AdvancedStatisticalComputing/Bios8366/data/textbook/baseball.dat", sep='\s+')
    predictors = baseball.copy()
    logsalary = predictors.pop('salary').apply(np.log)

    nrows, ncols = predictors.shape

    aic_values = []
    solution_current = solution_best = np.random.binomial(1, 0.5, ncols).astype(bool)
    solution_vars = predictors[predictors.columns[solution_current]]
    g = pd.ols(y=logsalary, x=solution_vars)
    aic_best = aic(g)
    aic_values.append(aic_best)

    # Cooling schedule
    tau = [tau_start * 0.9**i for i in range(periods)]
    for j in range(periods):

        for i in range(cooling[j]):

            # Random change n-neighborhood
            flip = np.random.choice(ncols,numChangeNeigh,replace = False)
            for i in range(numChangeNeigh):
                solution_current[flip[i]] = not solution_current[flip[i]]
            solution_vars = predictors[predictors.columns[solution_current]]
            g = pd.ols(y=logsalary, x=solution_vars)
            aic_step = aic(g)
            alpha = min(1, np.exp((aic_values[-1] - aic_step)/tau[j]))

            if ((aic_step < aic_values[-1]) or (np.random.uniform() < alpha)):
                # Accept proposed solution
                aic_values.append(aic_step)
                if aic_step < aic_best:
                    # Replace previous best with this one
                    aic_best = aic_step
                    solution_best = solution_current.copy()
            else:
                # Revert solution
                for i in range(numChangeNeigh):
                    solution_current[flip[i]] = not solution_current[flip[i]]
                aic_values.append(aic_values[-1])
    return aic_values,aic_best,solution_best


    a) Compare the effects of different cooling schedules (different temperatures and different durations at each temperature).

In [ ]:
#Spend more time at the low temperatures and start from a low temperature
periods = 15
numChangeNeigh = 1
#cooling
cooling_1 = [20* i for i in range(1,periods+1)] #[20 40 80 ... 16*20]
tau_start = 5
aic_values_1,aic_best_1,solution_best_1 = simAnnealing(periods,cooling_1,tau_start,numChangeNeigh)
plotResults(aic_values_1,aic_best_1,solution_best_1)

In [ ]:
# Spend much more time at the high temperature and start from high temperature
cooling_2 = np.fliplr([cooling_1])[0]
# inverse of cooling_1
tau_start = 80
aic_values_2,aic_best_2,solution_best_2 = simAnnealing(periods,cooling_2,tau_start,numChangeNeigh)
plotResults(aic_values_2,aic_best_2,solution_best_2)

In [ ]:
#Spend equal amount of time but start from medium temperature
cooling_3 = [120]*int(periods/3) + [120]*int(periods/3) + [120]*int(periods/3) #
tau_start = 50
aic_values_3,aic_best_3,solution_best_3 = simAnnealing(periods,cooling_3,tau_start,numChangeNeigh)
plotResults(aic_values_3,aic_best_3,solution_best_3)

The ossicilations of the aic_values increase as the algorithm spend more time on high temperatures and start from a relatively higher temperature. This behaviour is expected since higher temperature values increase the chances of accepting a value even if it is not the best so far. When we use a lower temperature, on the other hand, and spent more time at the lower temperatures, we see a much flatter-pattern where the aic_value does not change as much. The minimum valeu of the aic does not change a lot between different cooling scheduels.  

b) Compare the effect of a proposal distribution that is discrete uniform over 2-neighborhoods versus one that is discrete uniform over 3-neighborhoods.

In [ ]:
#Over 2-neighborhoods
numChangeNeigh = 2
tau_start = 20
aic_values_1,aic_best_1,solution_best_1 = simAnnealing(periods,cooling_1,tau_start,numChangeNeigh)
plotResults(aic_values_1,aic_best_1,solution_best_1)

In [ ]:
#Over 3-neighborhoods
numChangeNeigh = 3
aic_values_1,aic_best_1,solution_best_1 = simAnnealing(periods,cooling_1,tau_start,numChangeNeigh)
plotResults(aic_values_1,aic_best_1,solution_best_1)

As we can see from the plots above, the using uniform 3 neighborhoods create bigger oscillation between aic_values than 2 neighborhoods. There is not that much differnce in between besides how much the aic valus change.  

2. Implement a genetic algorithm for minimizing the AIC for the baseball salary regression problem. Model your algorithm on Example 3.5. 

In [ ]:
def calculate_fitness(aic_values):
    P = len(aic_values)
    aic_rank = (-aic_values).argsort().argsort()+1.
    return 2.*aic_rank/(P*(P+1.))

def geneticAlgo(mutation_rate,pop_size,useFitnessDad):

    baseball = pd.read_table("/Users/Ahmet/Box Sync/Classes/Vanderbilt/AdvancedStatisticalComputing/Bios8366/data/textbook/baseball.dat", sep='\s+')
    predictors = baseball.copy()
    logsalary = predictors.pop('salary').apply(np.log)
    nrows, ncols = predictors.shape
    iterations = 100

    aic_best = []
    best_solution = []
    aic_history = []

    # Initialize genotype
    current_gen = np.random.binomial(1, 0.5, pop_size*ncols).reshape((pop_size, ncols))


    for i in range(iterations):

        # Get phenotype
        current_phe = [predictors[predictors.columns[g.astype(bool)]] for g in current_gen]
        # Calculate AIC
        current_aic = np.array([aic(pd.ols(y=logsalary, x=x)) for x in current_phe])
        # Get lowest AIC
        aic_best.append(current_aic[np.argmin(current_aic)])
        best_solution.append(current_gen[np.argmin(current_aic)])

        # Calculate fitness according to AIC rank
        fitness = calculate_fitness(current_aic)

        # Choose first parents according to fitness
        moms = np.random.choice(range(pop_size), size=int(pop_size/2), p=fitness)
        # Choose second parents randomly
        if useFitnessDad:
            dads = np.random.choice(range(pop_size), size=int(pop_size/2),p = fitness)
        else:
            dads = np.random.choice(range(pop_size), size=int(pop_size/2))
                

        next_gen = []
        for x,y in zip(current_gen[moms], current_gen[dads]):
            # Crossover
            cross = np.random.randint(0, ncols)
            child1 = np.r_[x[:cross], y[cross:]]
            child2 = np.r_[y[:cross], x[cross:]]
            # Mutate
            m1 = np.random.binomial(1, mutation_rate, size=ncols).astype(bool)
            child1[m1] = abs(child1[m1]-1)
            m2 = np.random.binomial(1, mutation_rate, size=ncols)
            child2[m2] = abs(child1[m2]-1)
            next_gen += [child1, child2]

        # Increment generation
        current_gen = np.array(next_gen)
        # Store AIC values
        aic_history.append(current_aic)
    return aic_best

 1. Compare the effects of using different mutation rates.  

In [ ]:
# Mutation rate = 0.001, pop size = 30, dont use fitness for dad
aic_best = geneticAlgo(0.001,30,0)
plt.plot(aic_best,color='r')

In [ ]:
# Mutation rate = 0.1, pop size = 30, dont use fitness for dad
aic_best = geneticAlgo(0.1,30,0)
plt.plot(aic_best,color='r')

Increasing the mutation rate causes the aic value to change more drastically as the algorithm iterates. This behaviour is expected since mutation rate basically increses the change in the genetic code that will help the algorithm to explore more possible genetic codes. It will also increases the chances of moving away from finding the best aic value

 2. Compare the effects of using different generation sizes.  

In [ ]:
# Mutation rate = 0.01, pop size = 10, dont use fitness for dad
aic_best = geneticAlgo(0.01,10,0)
plt.plot(aic_best,color='r')

In [ ]:
# Mutation rate = 0.01, pop size = 100, dont use fitness for dad
aic_best = geneticAlgo(0.01,100,0)
plt.plot(aic_best,color='r')

We can say that the effect of increasing the population size is similiar to increasing mutation rate.  

3. Instead of the selection mechanism used in the class example, try using independent selection of both parents with probabilities proportional to their fitness.

In [ ]:
# Mutation rate = 0.01, pop size = 20,  use fitness for dad
aic_best = geneticAlgo(0.01,20,1)
plt.plot(aic_best,color='r')

Using fitness decreases exploration of possible solutions. 

## Question 3

Use the combinatorial optimization method of your choice to obtain a solution to the traveling salesman problem for the Brazilian cities described in the lecture notes, using minimum total distance as the criterion. Use the the first city listed in the dataset as "home" (*i.e.* the trip must start and end there. I will award 5 bonus points to the best solution!

In [ ]:
from copy import deepcopy
import math

def parse_latlon(x):
    d, m, s = map(float, x.split(':'))
    ms = m/60. + s/3600.
    if d<0:
        return d - ms
    return d + ms

def swapCities(cities, nSwap):
    if nSwap % 2 != 0:
        nSwap -= 1
    nSwap = int(max(nSwap, 2))
    nSwap = int(nSwap / 2)
    citiesToSwap = np.random.choice(len(cities) - 2,nSwap*2,replace=False)
    while citiesToSwap[0] == 0 or citiesToSwap[1] == 0:
        citiesToSwap = np.random.choice(len(cities) - 1,nSwap*2,replace=False)
    for i in range(nSwap):
        tempCity = np.copy(cities[citiesToSwap[i*2]])
        cities[citiesToSwap[i*2]] = cities[citiesToSwap[i*2+1]]
        cities[citiesToSwap[i*2+1]] = tempCity

    return cities

def calculateTotalDistance(cities):
    dist = 0
    for i in range(len(cities) - 1):
        dist += math.sqrt((cities[i][1] - cities[i + 1][1]) ** 2 + (cities[i][2] - cities[i + 1][2]) ** 2)
    return dist


cities =  pd.read_csv('../data/brasil_capitals.txt', 
                      names=['city','lat','lon'])[['lat','lon']].applymap(parse_latlon)

beginTemp = 10
endTemp = 0.001
coolingFactor = 0.9
period = 840
bestDistances = []
cityArray = np.transpose(np.array((range(len(cities)), cities['lat'], cities['lon']), np.float64))
numSwap = 2
bestCityOrder = np.random.permutation(cityArray)
bestCityOrder = np.vstack([bestCityOrder,bestCityOrder[0]])
bestDistance = calculateTotalDistance(bestCityOrder)
bestDistances.append(bestDistance)

for j in range(period):

    temperature = beginTemp

    currentDistance = bestDistance
    currentCities = np.copy(bestCityOrder)

    newCities = np.copy(currentCities)
    newDistance = currentDistance



    while True:
        newCities = swapCities(newCities, numSwap)
        newDistance = calculateTotalDistance(newCities)

        if newDistance < currentDistance or math.exp(
                        (currentDistance - newDistance) / temperature) > np.random.random():
            currentDistance = newDistance
            currentCities = np.copy(newCities)
        else:
            newDistance = currentDistance
            newCities = currentCities[:]
        if newDistance < bestDistance:
            bestCityOrder = deepcopy(newCities)
            bestDistance = newDistance
            bestDistances.append(bestDistance)
        temperature *= coolingFactor
        #numSwap = int(numSwap * coolingFactor)
        if temperature < endTemp:
            break

#print(bestDistances)
print('best distance = ',min(bestDistances))
#print(bestCityOrder)
plt.plot(bestCityOrder[:,1],bestCityOrder[:,2],'ro',bestCityOrder[:,1],bestCityOrder[:,2],'b')



## Question 4

Suppose $y$ has a binomial distribution with parameters $n$ and $p$, and we are interested in the log-odds value $\theta = \log(p/(1 − p))$. Our prior for $\theta$ is that $\theta \sim N(\mu, \sigma^2)$. It follows that the posterior density of $\theta$ is given, up to a proportionality constant, by:

$$g(\theta | y) \propto \frac{\exp(y\theta)}{(1 + exp(\theta))^n} \exp\left[\frac{-(\theta − \mu)^2}{2\sigma^2}\right]$$

For example, suppose we are interested in learning about the probability that a possibly-biased coin lands heads when tossed. *A priori* we believe that the coin is fair, so we assign $\theta$ a $N(0,.25)$ prior. We toss the coin $n = 5$ times and obtain $y = 5$ heads.

1. Using a normal approximation to the posterior density, compute the probability that the coin is biased toward heads (i.e., that θ is posi- tive).

In [ ]:
from scipy.stats import norm
from scipy.optimize import fmin_bfgs
def posterior(param, n, y):

    theta = param
    post = y*theta
    post -= n*np.log(1+np.exp(theta))
    post -= ((theta-mu)**2)/(2*sigma)

    return post

def calc_diff(theta, n, y):

    return posterior(theta, n, y) - np.log(norm.pdf(theta,mu,sigma))

calc_diff_min = lambda *args: -calc_diff(*args)

posterior_min = lambda *args: -posterior(*args)

mu = 0
sigma = 0.25
init_value = (0.1)
n= 5
y = 5

opt = fmin_bfgs(posterior_min, init_value,
          args=(n, y), full_output=True)
mode, var = opt[0], opt[3]

prob = 1 - norm.cdf(0,mode,var)
print(prob)

2. Using the prior density as a proposal density, design a rejection algo- rithm for sampling from the posterior distribution. Using simulated draws from your algorithm, approximate the probability that the coin is biased toward heads.

In [ ]:
def reject(post, n, data, c):

    k = len(mode)

    # Draw samples from g(theta)
    theta = norm.rvs(mu, sigma, size=n)

    # Calculate probability under g(theta)
    gvals = np.array([np.log(norm.pdf(t, mu, sigma)) for t in theta])

    # Calculate probability under f(theta)
    fvals = np.array([post(t, data[0], data[1]) for t in theta])

    # Calculate acceptance probability
    p = np.exp(fvals - gvals + c)

    return theta[np.random.random(n) < p]

c_init = 0.8
opt = fmin_bfgs(calc_diff_min,c_init,
                args=(5, 5),
                full_output=True)

print('c values = ',opt[1])
prob = reject(posterior,1e4,[n,y],opt[1])

counter = 0
for i in range(len(prob)):
    if prob[i] > 0:
        counter+=1
print('Probability of heads = ',counter/prob.size)

3. Using the prior density as a proposal density, simulate values from the posterior distribution using the SIR algorithm. Approximate the probability that the coin is biased toward heads.

In [ ]:
sampSize= 10000

normalSamples = norm.rvs(mu, sigma, size=sampSize)

f_theta = np.array([posterior(t, n, y) for t in normalSamples])

q_theta = np.array([np.log(norm.pdf(t, mu, sigma)) for t in normalSamples])

w = np.exp(f_theta - q_theta - max(f_theta - q_theta))
p_sir = w/w.sum()
theta_sir = normalSamples[np.random.choice(range(len(normalSamples)), size=10000, p=p_sir)]
counter = 1
for i in range(len(theta_sir)):
    if theta_sir[i] > 0:
        counter+=1
print(counter/len(theta_sir))